# Calligaro nicolas
## Création des données sur la course
Readme :\
Ce jupyter se concentre sur récupérer les données du classement et produire un DataFrame exploitable.\
Runner l'ensemble du jupyter. vérifier que la dernière cellule et ok (si erreur lors du dl des fichiers xls relancer le jupyter)\
la dernière cellule est le 'main'

In [1]:
from os import listdir,mkdir
import pandas as pd
import re
import pickle
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import mptools
header = ["Rang","Nat. / Voile","Skipper / Bateau","Heure FR","Latitude","Longitude","1 Cap","1 Vitesse","1 VMG"
      ,"1 Distance","2 Cap","2 Vitesse","2 VMG","2 Distance","3 Cap","3 Vitesse","3 VMG","3 Distance","DTF","DTL"]

In [2]:
def prod_xls(html):
    """
    Fonction qui va télécharger en produire les fichiers xls dans le dossier temp
    voir le fichier mptools.py qui contient le code du multiprocessing
    """
    
    check=0
    urls=[]
    r = requests.get(f'{html}/fr/classement')
    soup = BeautifulSoup(r.content)
    for tag in soup.findAll('option'):
        if tag.attrs['value'] :
            urls.append(f"{html}/download-race-data/vendeeglobe_{tag.attrs['value']}.xlsx")
    check = len(urls)
    if __name__ ==  '__main__': 
        with Pool() as p :
            done = p.map(mptools.Get_Xlsx,[url for url in urls])
    for i in done :
        if not i:
            check-=1
    print (f"on a traité {check} fichiers / sur {len(urls)} prévus")
    return

In [3]:
def prod_df (fichier):
    """
        On ouvre le fichier passé en entré
        On traite le header et les ligne en trop (en haut et en bas)
        On produit notre propre entête en retirant celle du fichier (on pourrait faire mieux)
        On récupère la date et l'heure de l'extraction dans le nom du fichier
        On produit 3 nouvelles colonnes :
            1 contenant la date (sans l'heure)
            2 contenant l'heure (sans la date)
            3 contenant la date complète (jour, heure)
        
    """
    raw_df = pd.read_excel (f"temp/{fichier}"
                            ,header = None
                            ,skiprows = 5
                            ,names = header
                            ,skipfooter = 4    
                            ,na_values = ['RET','NL']
                          )
    heure_fichier = fichier[:-5]
    if heure_fichier == '20201108_140000':#On attrape le 1er xls
        #On donne a tout le monde le bon point de départ
        raw_df['Longitude']= raw_df['Longitude'].fillna(method='ffill')
        raw_df['Latitude']= raw_df['Latitude'].fillna(method='ffill')
        #On donne a tout le monde la bonne distance parcourus de départ
        raw_df['2 Distance']= "0.0 nm"
    #On rajoute la date complète lié au nom du fichier
    raw_df.insert (3,'Date full',pd.to_datetime(heure_fichier, format = '%Y%m%d_%H%M%S',errors = 'coerce'))
    return raw_df

In [4]:
def prepare_df (df):
    """
        On travail sur les colonnes du DF (voir chaque commentaire)
    """
    #la colone heure est inutile.
    df.drop(['Heure FR'], axis = 1, inplace = True)
    #On subdivise la colonne Nat / voile en deux colonnes. puis on retire la dite colonne
    #On fera attention le champ peut etre en 2 ou 3 partie
    df.insert (2,'Nationalité',df[df.columns[1]].apply(lambda x : str(x.split()[-2:-1][0])))
    df.insert (3,'Num_voile',df[df.columns[1]].apply(lambda x : int(x.split()[-1:][0])))
    df.drop(['Nat. / Voile'], axis = 1, inplace = True)
    #on subdivise la colone Skipper / bateau en deux colonnes. puis on retire la dite colonne.
    df.insert (4,'Skipper',df[df.columns[3]].apply(lambda x : str(x.split('\n')[0]).replace('é','e').lower()))
    #On créé la colonne clé qui contient le nom de famille pour la jointure des 2 DF
    df['Cle']=df['Skipper'].apply(lambda x: x.split(' ')[-1])
    df.insert (5,'Bateau',df[df.columns[3]].apply(lambda x : str(x.split('\n')[1])))
    df.drop(['Skipper / Bateau'], axis = 1, inplace = True)
    
    
    #On trie le DF par numéro de voile et par les dates.
    df = df.sort_values(by = ['Num_voile','Date full'])
    #On remplace tous les na trouvé et récupérant la valeur du dessus
    #NB : toute les voiles ont eu une valeur de départ
    #Pour les bateaux retiré, on considère qu'il reste sur place
    df['Longitude']= df['Longitude'].fillna(method='pad')
    df['Latitude']= df['Latitude'].fillna(method='ffill')
    #On change le modèle des coordonées
    df['Latitude'] = df['Latitude'].apply(lambda x : coord(x))
    df['Longitude'] = df['Longitude'].apply(lambda x : coord(x))
    #On retire les lignes avec un NA
    df= df.dropna()
    #On passe la colone Rang en numérique
    df['Rang'] = pd.to_numeric(df['Rang'])
    df = df.astype({"Rang": int})#on force le type int au lieu de float
    #On retire toutes les unités de mesure et on cast en float
    for i in list(df.columns)[8:-1]:
        df[i] = df[i].apply(lambda x : get_out_unit(x))
    return df

In [5]:
def coord(champ):
    """
        On convertie le champ long et lat en coord gps
        avec la formule :
        W/S = -1
        X°Y'Z"= X+(Y/60)+(Z/3600)
    """
    
    if str(champ) == 'nan' :
        #print("Na détecté")
        return False
    else : 
        #print (f"{champ}")
        #print (f"{int(champ[:2])} {int(champ[3:5])/60:.3} {int(champ[6:8])/3600:.3}")
        degree = int(champ[:2])
        minute = int(champ[3:5])/60
        seconde = int(champ[6:8])/3600
        if champ[-1] == 'S' or champ[-1] == 'W' :
            card =  -1
        else :
            card = 1
    return float(card * (degree + minute + seconde))
    

In [6]:
def get_out_unit(text):
    """
    On filtre et ne rend que les valeurs numériques
    Cela retire toutes les unités de mesure, pour avoir des champs numérique
    """
    masque = "[0-9]*[.]*[0-9]"
    return float(re.compile(masque).search(text).group())

In [7]:
def get_df_classement(html):
    """
    On produit un DF vierge avec la bonne entête.
    On lit chaque fichier, on en produit un DF
    On les concatène entre eux
    On les nettoye
    On retourne de DF du Classement
    Le DF sera mis en mémoire et appeler a chaque début de jupyter
    """
    #On créé le dossier temp si possible.
    try :
        mkdir('temp')
    except FileExistsError:
        pass
    #On télécharge tous les fichiers excels
    prod_xls(html)
    #on créer un DF vide mais avec les bonnes colonnes.
    df = pd.DataFrame(columns=header)
    #On parcours chaque fichier du dossier
    for file in listdir(f"temp/"):
        #On produit un DF brut avec quelques modifications
        df1 = prod_df(file)
        #On concatène l'ensemble des DF en un seul
        df = df1.append(df)
    #On nettoye les données
    df = prepare_df(df)
    #On sauvegarde le DF classement comme objet pour l'utiliser ailleurs
    pickle.Pickler(open("df_clas.pkl",'wb')).dump(df)
    return df


In [8]:
df_clas= get_df_classement('https://www.vendeeglobe.org')

on a traité 129 fichiers / sur 129 prévus


In [9]:
df_clas.shape#,df_clas.info()

(4142, 23)